In [1]:
# Importing Necessary Libraries
import cv2
import os
import shutil
import math
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [2]:
def image_segmentation(original_img):

#     padded_image=cv2.copyMakeBorder(original_img,1,1,1,1,cv2.BORDER_CONSTANT,value=(0,0,0))
    J=cv2.bilateralFilter(original_img,9,75,75)

#     a=np.array(J)
#     m,n,p=a.shape
#     for r in range(1,m-1):
#         for c in range(1,n-1):
#             s=J[r,c]
#             n1=J[r+1,c]
#             n2=J[r-1,c]
#             n3=J[r,c+1]
#             n4=J[r,c-1]
#             n5=J[r-1,c-1]
#             n6=J[r+1,c+1]
#             n7=J[r+1,c-1]
#             n8=J[r-1,c+1]

#             s1=s-n1
#             s2=s-n2
#             s3=s-n3
#             s4=s-n4
#             s5=s-n5
#             s6=s-n6
#             s7=s-n7
#             s8=s-n8
#             threshold=150
#             if (all(s1<threshold) and all(s2<threshold) and all(s3<threshold) and all(s4<threshold) and all(s5<threshold) and all(s6<threshold) and all(s7<threshold) and all(s8<threshold)):
#                 J[r,c]=0;
#             else:
#                 J[r,c]=s;

#     ret, final_img = cv2.threshold(J, 110, 255, cv2.THRESH_BINARY_INV)
    return J;
# Function for Formatting Dataset

In [3]:
pip install kaggle

Note: you may need to restart the kernel to use updated packages.


In [4]:
! pip install -q kaggle
import os
if not os.path.isfile(os.path.expanduser('~/.kaggle/kaggle.json')):
  from google.colab import files
  print("Upload kaggle.json here")
  files.upload()

if not os.path.isfile('IMDB Dataset.csv'):
  !mkdir ~/.kaggle
  !mv ./kaggle.json ~/.kaggle/
  !chmod 600 ~/.kaggle/kaggle.json

  dataset_name = 'prahladmehandiratta/cervical-cancer-largest-dataset-sipakmed'
  zip_name = dataset_name.split('/')[-1]

  !kaggle datasets download -d {dataset_name}
  !unzip -q ./{zip_name}.zip -d .

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
import zipfile
import os

# Path to the zip file
zip_file_path = '/content/cervical-cancer-largest-dataset-sipakmed.zip'  # Replace with the actual path to your zip file

# Directory to extract the contents
extracted_folder_path = '/content/cerival/'  # Replace with the desired extraction path

# Create the target directory if it doesn't exist
os.makedirs(extracted_folder_path, exist_ok=True)

# Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

# List the contents of the extracted folder
extracted_contents = os.listdir(extracted_folder_path)

# Print the list of extracted contents
print("Extracted contents:", extracted_contents)


Extracted contents: ['im_Metaplastic', 'im_Parabasal', 'im_Koilocytotic', 'im_Superficial-Intermediate', 'im_Dyskeratotic']


In [ ]:
# Function for Formatting Dataset
def FormatDataset(dataset_src, dataset_dest, classes):
    # Making a Copy of Dataset
    new_cropped_dest = [os.path.join(dataset_dest, cls, 'CROPPED') for cls in classes];
    new_complete_dest = [os.path.join(dataset_dest, cls, 'COMPLETE') for cls in classes];
    cropped_src = [ dataset_src + "/im_" + cls + "/im_" + cls + "/CROPPED" for cls in classes ];
    complete_src = [ dataset_src + "/im_" + cls + "/im_" + cls for cls in classes ];
    for (dest1, dest2) in zip(new_cropped_dest, new_complete_dest):
        os.makedirs(dest1);
        os.makedirs(dest2);
    # Formating Cropped Images
    for (src,new_dest) in zip(cropped_src, new_cropped_dest):
        for file in os.listdir(src):
            filename, file_ext = os.path.splitext(file);
            if file_ext == '.bmp':
                img_des = os.path.join(new_dest, filename + '.jpg');
                img = cv2.imread(os.path.join(src, file));
                img = cv2.resize(img, (75, 75));
                img = cv2.copyMakeBorder(img, 1, 1, 1, 1, cv2.BORDER_CONSTANT, value=0);
                img = cv2.blur(img, (2, 2));
                img = image_segmentation(img);
                cv2.imwrite(img_des ,img);

# Source Location for Dataset
src = '/content/cerival';
# Destination Location for Dataset
dest = './CervicalCancer';
# Image Classes
classes = ["Dyskeratotic","Koilocytotic","Metaplastic","Parabasal","Superficial-Intermediate"];
# Formatting Dataset
FormatDataset(src, dest, classes);

In [ ]:
root_dir = "./CervicalCancer"
classes = ["Dyskeratotic","Koilocytotic","Metaplastic","Parabasal","Superficial-Intermediate"]

def GetDatasetSize(path, classes, main = "CROPPED"):
    num_of_image = {}
    for cls in classes:
        # Counting the Number of Files in the Folder
        num_of_image[cls] = len(os.listdir(os.path.join(path, cls, main)));
    return num_of_image;

print(GetDatasetSize(root_dir, classes, "COMPLETE"));

{'Dyskeratotic': 0, 'Koilocytotic': 0, 'Metaplastic': 0, 'Parabasal': 0, 'Superficial-Intermediate': 0}


In [ ]:
def TrainValTestSplit(root_dir, classes_dir, main = "CROPPED", val_ratio = 0.15, test_ratio = 0.15):
    for cls in classes_dir:
        # Creating Split Folders
        os.makedirs('train/' + cls)
        os.makedirs('val/' + cls)
        os.makedirs('test/' + cls)

        # Folder to copy images from
        src = os.path.join(root_dir, cls, main);

        # Spliting the Files in the Given ratio
        allFileNames = os.listdir(src)
        np.random.shuffle(allFileNames)
        train_FileNames, val_FileNames, test_FileNames = np.split(np.array(allFileNames), [int(len(allFileNames)* (1 - (val_ratio + test_ratio))), int(len(allFileNames)* (1 - test_ratio))])

        train_FileNames = [src+'/'+ name for name in train_FileNames.tolist()]
        val_FileNames = [src+'/' + name for name in val_FileNames.tolist()]
        test_FileNames = [src+'/' + name for name in test_FileNames.tolist()]

        # Printing the Split Details
        print(cls,':')
        print('Total images: ', len(allFileNames))
        print('Training: ', len(train_FileNames))
        print('Validation: ', len(val_FileNames))
        print('Testing: ', len(test_FileNames))

        # Copy-pasting images
        for name in train_FileNames:
            shutil.copy(name, 'train/' + cls)

        for name in val_FileNames:
            shutil.copy(name, 'val/' + cls)

        for name in test_FileNames:
            shutil.copy(name, 'test/' + cls)
        print();


# Preforming Train / Validation / Test Split
root_dir = "./CervicalCancer"               # Dataset Root Folder
classes_dir = ["Dyskeratotic", "Koilocytotic", "Metaplastic", "Parabasal", "Superficial-Intermediate"]   # Classes
TrainValTestSplit(root_dir, classes_dir);

Dyskeratotic :
Total images:  813
Training:  569
Validation:  122
Testing:  122

Koilocytotic :
Total images:  825
Training:  577
Validation:  124
Testing:  124

Metaplastic :
Total images:  793
Training:  555
Validation:  119
Testing:  119

Parabasal :
Total images:  787
Training:  550
Validation:  118
Testing:  119

Superficial-Intermediate :
Total images:  831
Training:  581
Validation:  125
Testing:  125



In [ ]:
# Importing Keras for Image Classification
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.layers import Dense,Conv2D, Flatten, MaxPool2D, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import image
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model

In [ ]:
# CNN Model

model = Sequential()

# Convolutional Layer with input shape (64,64,3)
model.add(Conv2D(filters=32, kernel_size= (3,3), activation= 'relu', input_shape=(75,75,3)) )

model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu' ))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu' ))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu' ))
model.add(MaxPool2D(pool_size=(2,2)))



model.add(Flatten())
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=5, activation='sigmoid'))

#model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.002),
              #loss='binary_crossentropy', metrics=['accuracy'])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']  )

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 73, 73, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 71, 71, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2  (None, 35, 35, 32)        0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 33, 33, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 16, 16, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_3 (Conv2D)           (None, 14, 14, 128)       7

In [ ]:
# Expand the size of dataset with new transformed images from the original dataset using ImageDataGenerator.
train_datagen = image.ImageDataGenerator(zoom_range = 0.2, shear_range = 0.2 , rescale = 1./255 , horizontal_flip=True)
val_datagen = image.ImageDataGenerator(rescale = 1./255)
test_datagen = image.ImageDataGenerator(rescale = 1./255)

In [ ]:
# Expand the size of dataset with new transformed images from the original dataset using ImageDataGenerator.
train_datagen = image.ImageDataGenerator(zoom_range = 0.2, shear_range = 0.2 , rescale = 1./255 , horizontal_flip=True)
val_datagen = image.ImageDataGenerator(rescale = 1./255)
test_datagen = image.ImageDataGenerator(rescale = 1./255)
train_data = train_datagen.flow_from_directory(directory= "./train", target_size=(75, 75), batch_size=100, class_mode = 'categorical')

Found 2832 images belonging to 5 classes.


In [ ]:
train_data.class_indices

{'Dyskeratotic': 0,
 'Koilocytotic': 1,
 'Metaplastic': 2,
 'Parabasal': 3,
 'Superficial-Intermediate': 4}

In [ ]:
val_data = val_datagen.flow_from_directory(directory= "./val", target_size=(75, 75), batch_size=100, class_mode = 'categorical')


Found 608 images belonging to 5 classes.


In [ ]:
test_data = test_datagen.flow_from_directory(directory= "./test", target_size=(75, 75), batch_size=100, class_mode = 'categorical')

Found 609 images belonging to 5 classes.


In [ ]:
# Adding Model check point Callback
mc = ModelCheckpoint(filepath="cervical_cancer_best_model.hdf5", monitor= 'val_accuracy', verbose= 1, save_best_only= True, mode = 'auto');
call_back = [ mc ];

In [ ]:
# Fitting the Model
cnn = model.fit(
    train_data,
    steps_per_epoch = train_data.samples//train_data.batch_size,
    epochs = 30,
    validation_data = val_data,
    validation_steps = val_data.samples//val_data.batch_size,
    callbacks = call_back
    )

Epoch 1/30
28/28 [==============================] - ETA: 0s - loss: 1.1178 - accuracy: 0.5673
Epoch 1: val_accuracy improved from -inf to 0.60333, saving model to cervical_cancer_best_model.hdf5
28/28 [==============================] - 56s 2s/step - loss: 1.1178 - accuracy: 0.5673 - val_loss: 1.0653 - val_accuracy: 0.6033
Epoch 2/30
28/28 [==============================] - ETA: 0s - loss: 0.9803 - accuracy: 0.6384
Epoch 2: val_accuracy improved from 0.60333 to 0.65500, saving model to cervical_cancer_best_model.hdf5
28/28 [==============================] - 52s 2s/step - loss: 0.9803 - accuracy: 0.6384 - val_loss: 0.9283 - val_accuracy: 0.6550
Epoch 3/30
28/28 [==============================] - ETA: 0s - loss: 0.8028 - accuracy: 0.7130
Epoch 3: val_accuracy improved from 0.65500 to 0.73667, saving model to cervical_cancer_best_model.hdf5
28/28 [==============================] - 54s 2s/step - loss: 0.8028 - accuracy: 0.7130 - val_loss: 0.7081 - val_accuracy: 0.7367
Epoch 4/30
28/28 [====

In [ ]:
model.save('cerival_cnn.h5')

In [ ]:
import cv2
import numpy as np
from keras.preprocessing import image
from PIL import Image

# Function to load and preprocess the image
def load_and_preprocess_image(image_path):
    # Load and preprocess the new image
    img = Image.open(image_path)
    img = img.resize((75, 75))  # Adjust the target size to match your model's input size
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize pixel values to the range [0, 1]

    return img_array

# Example usage:
new_image_path = '/content/CervicalCancer/Superficial-Intermediate/CROPPED/001_03.jpg'
new_image = load_and_preprocess_image(new_image_path)

# Ensure the input shape matches the expected input size of the model
if new_image.shape[1:] != (75, 75, 3):
    raise ValueError(f"Expected image data of shape (1, 75, 75, 3), but got {new_image.shape}")

# Assuming you have a loaded model named 'model'
predictions = model.predict(new_image)

# Process the predictions as needed for your application
print(predictions)
predicted_class_index = np.argmax(predictions)
print("Predicted Class Index:", predicted_class_index)



1/1 [==============================] - 0s 25ms/step
[[0.48285806 0.8294237  0.1254598  0.00318632 0.9941982 ]]
Predicted Class Index: 4


Predicted Class Index: 0
